In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time

import numpy as np
from numpy.typing import NDArray
from typing import List, Tuple
import os, os.path

number_run = 1

In [ ]:

for t in range(number_run):
    run = f"run2_trial3_issuance_1000"
    print(run)
    DIR = f"../library_test/credentials/{run}"
    files = [name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]
    files.sort()

    for name in files:
        df = pd.read_csv(os.path.join(DIR, name))
        df.to_csv(f"cleanup_{run}.csv", mode= "a", header= (not os.path.exists(f"cleanup_{run}.csv")), index = False)
        print(name)


In [ ]:
file_name = "run2_trial3_issuance_1000"

df = pd.read_csv(f"cleanup_{file_name}.csv")
df["New_Proof"] = df["proof array"].apply(lambda x: list(map(int, x.split(','))))
df["Size"] = df["New_Proof"].apply(lambda x: len(x))
max_length = df["Size"].max()
df["number_attributes"]= 12+df["Number of general additional informations"]+3*df["Number of categories"]+df["Number of additional inforations of each categories"].apply(lambda x: sum(map(int, x.split(','))))
# Number of categories,
# Number of additional inforations of each categories,
# Number of general additional informations

print("Max length: ", max_length)
for x in range(max_length):
    df["proof_array_"+str(x)] = df["New_Proof"].apply(lambda y: y[x] if len(y) > x else 0)
df = df.drop("New_Proof", axis= 1)
df = df.drop("proof array", axis= 1)
df.to_csv(f"formatted_{file_name}.csv", index=False)
print(f"{file_name}.csv end")

In [ ]:
df_total = pd.read_csv(f"cleanup_run1_trial3_issuance_100.csv").iloc[:600000].copy()

In [ ]:
df_total